In [2]:
from pymorphy2 import MorphAnalyzer
import nltk

In [3]:
from nltk.tokenize import word_tokenize
from string import punctuation

### Ручная разметка корпуса
Был взят текст отзыва на фильм (несколько видоизменен).
#### Почему текст подходит для оценки?
* есть аббревиатуры
* иностранные имена персонажей и актеров, написанные в русской орфографии, у них сложно определить часть речи
* встречаются слова, имеющие омонимы других частей речи ("довольно": прил. ср.р./нар./част.)
* есть числительные, частично написанные цифрами ("80-ые", "90-ые")
* сленговые слова, в разных вариантах написания ("экшн", "экшен")
* заимствованные слова, относительно новые/незнакомые (характерны для вселенной фильма) для русского языка ("сиквел", "симбиот")
#### Комментарий к тегсету ручной разметки
За основу брался тегсет pymorphy, но был несколько упрощен, например, разные виды прилагательных (краткие и полные) сводились к одному. Это было сделано для удобства тестирования других теггеров, где подобные случаи решаются по-разному (все прил. размечаются как adj вне зависимости от их краткости)

In [6]:
df = pd.read_csv("my_sample_data.csv", sep = ';')


In [119]:
data_list = []
with open ("my_sample_data.csv", encoding = 'utf-8') as f1:
    text1 = f1.readlines()
    for line in text1:
        line = line.split(';')
        wordm = line[1]
        if '\n' in line[2]:
            second = line[2][0:-1]
        else:
            second = line [2]
        if line[1].endswith(' '):
            wordm = line[1][0:-1]
        line = (wordm, second)
        data_list.append (line)
print(data_list)

[('Люди', 'NOUN'), ('росшие', 'PRTF'), ('в', 'PREP'), ('80-ые', 'NUMR'), ('и', 'CONJ'), ('90-ые', 'NUMR'), ('знакомы', 'ADJ'), ('с', 'PREP'), ('именами', 'NOUN'), ('из', 'PREP'), ('заголовка', 'NOUN'), ('не', 'PRCL'), ('понаслышке', 'ADV'), ('Веном', 'NOUN'), ('и', 'CONJ'), ('Карнадж', 'NOUN'), ('были', 'VERB'), ('одними', 'NUMR'), ('из', 'PREP'), ('самых', 'PRON'), ('ярких', 'ADJ'), ('и', 'CONJ'), ('запоминающихся', 'PRTF'), ('злодеев', 'NOUN'), ('из', 'PREP'), ('мультсериала', 'NOUN'), ('про', 'PREP'), ('Человека-Паука', 'NOUN'), ('В', 'PREP'), ('комиксах', 'NOUN'), ('же', 'PRCL'), ('это', 'PRON'), ('в целом', 'PREP'), ('довольно', 'ADV'), ('важный', 'ADJ'), ('персонаж', 'NOUN'), ('который', 'CONJ'), ('за', 'PREP'), ('многие', 'ADJ'), ('десятки', 'NOUN'), ('лет', 'NOUN'), ('своего', 'PRON'), ('существования', 'NOUN'), ('пережил', 'VERB'), ('множество', 'NOUN'), ('различных', 'ADJ'), ('сюжетов', 'NOUN'), ('Неудивительно', 'ADV'), ('что', 'CONJ'), ('именно', 'PRCL'), ('Веномом', 'NOUN'

### 1. pymorphy

In [41]:
morph = MorphAnalyzer()


In [122]:
with open ('Текст_корпус.txt', encoding = 'utf-8') as f:
    text = f.read()
    tokenizer = word_tokenize(text)
    for elem in tokenizer:
        if elem in punctuation or elem.isalpha == False:
            tokenizer.remove(elem)

In [123]:
my_pymorph_list = []
for word in tokenizer:
    morph_data = str(morph.parse(word)[0].tag.POS)
    if morph_data != '':
        if morph_data == 'NPRO':
            morph_data = 'PRON'
        if morph_data == 'INFN' or morph_data == 'PRTF' or morph_data == 'PRTS': #адаптируем обозначения частей речи под разметку текста нашего корпуса
            morph_data = 'VERB'
        if morph_data.startswith ('ADJ'):
            morph_data = 'ADJ'
        if morph_data == 'ADVB':
            morph_data = 'ADV'
        pos_tuple = (word, morph_data)
        my_pymorph_list.append(pos_tuple)
len(my_pymorph_list)

317

In [124]:
same_counter = 0 #подсчет количества совпадений с ручной разметкой
for some in my_pymorph_list:
    if some in data_list:
        same_counter += 1
    else:
        pass
same_counter

259

#### Подсчет accuracy для pymorphy:

In [121]:
acc = same_counter/len(data_list)
acc

0.590443686006826

### 2. Natasha

In [78]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)


segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)


In [67]:
with open ('Текст_корпус.txt', encoding = 'utf-8') as f1:
    text = f1.read()
doc = Doc (text)

In [70]:
doc.segment(segmenter)
print(doc.tokens[:5])


[DocToken(stop=4, text='Люди'), DocToken(start=4, stop=5, text=','), DocToken(start=6, stop=12, text='росшие'), DocToken(start=13, stop=14, text='в'), DocToken(start=15, stop=20, text='80-ые')]


In [77]:
doc.tag_morph(morph_tagger)
print(doc.tokens[:5])
for elem in doc.tokens[:5]:
    print(elem.text, elem.pos)

[DocToken(stop=4, text='Люди', pos='NOUN', feats=<Anim,Nom,Masc,Plur>), DocToken(start=4, stop=5, text=',', pos='PUNCT'), DocToken(start=6, stop=12, text='росшие', pos='VERB', feats=<Perf,Nom,Plur,Past,Part,Act>), DocToken(start=13, stop=14, text='в', pos='ADP'), DocToken(start=15, stop=20, text='80-ые', pos='ADJ')]
Люди NOUN
, PUNCT
росшие VERB
в ADP
80-ые ADJ


In [80]:
nat_list = [] #создаем список кортежей, в каждом из которых первый элеметн - слово, второй - его часть речи
for elem in doc.tokens:
    if elem.pos != 'PUNCT':
        if elem.pos == 'NUM':
            pos_data = 'NUMR'
        if elem.pos == 'ADP':
            pos_data = 'PREP'
        if elem.pos == 'CCONJ':
            pos_data = 'CONJ'
        if elem.pos == 'PROPN':
            pos_data = 'NOUN'
        else:
            pos_data = elem.pos
        elem_data = (elem.text, pos_data)
        nat_list.append (elem_data)
len (nat_list)  

315

In [82]:
same_nat_counter = 0 #подсчет количества совпадений с результатами ручной разметки
for some in nat_list:
    if some in data_list:
        same_nat_counter += 1
    else:
        pass
same_nat_counter

173

#### Подчет accuracy для natasha

In [120]:
acc = same_nat_counter/len(data_list)
acc

0.590443686006826

### Функция chunker
Используем pymorphy, так как он показал лучший результат. Функция будет искать 3 наиболее частных биграммы с частицей "не" и возвращать части речи, следующие после этой частицы

In [115]:
from collections import defaultdict, Counter

In [116]:
def chunker (my_pymorph_list):
    pos_dict = defaultdict(int)
    for i in range (len(my_pymorph_list)):
        if my_pymorph_list[i][0]=='не':
            pos_for_func = my_pymorph_list[i+1][1]
            pos_dict[pos_for_func] += 1
    k = Counter (pos_dict)
    lead = k.most_common(3) #возвращает 3 наиболее частотных части речи, употребляемых после "не"
    return (lead)
chunker (my_pymorph_list)

[('VERB', 6), ('ADV', 1), ('CONJ', 1)]

### Варианты улучшения предыдущей домашки
Могу предложить следующие шаблоны, все три могут встретиться как в положительных, так и в отрицательных отзывах, также все три в принципе частотны в языке, поэтому с большой вероятностью встретятся в любой выборке отзывов: 
1. прил+сущ. (так как часто характеристики в отзывах даются в виде "характеристика+понятие", например, интересный сюжет, неинтересный персонаж)
2. не+глагол (для отзывов характерно употребление оценочных глаголов с не: не впечатлил, не оставляет равнодушным)
3. глагол+наречие (часто встречается что-то вроде "прекрасно описывает")
